In [1]:
import zipfile
import requests 
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder,PolynomialFeatures
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.model_selection import cross_val_score, cross_val_predict

In [163]:
df_train = pd.read_csv('train.csv')
df_train.fillna(0,inplace=True)
df_test = pd.read_csv('test.csv')
df_out = pd.DataFrame(columns=['NU_INSCRICAO','NU_NOTA_MT'])
df_out['NU_INSCRICAO'] = df_test['NU_INSCRICAO']
df_test.drop(['NU_INSCRICAO'],axis=1,inplace=True)
df_test.fillna(0,inplace=True)
cols = list(set(df_test.columns) & set(df_train.columns))
cols.append('NU_NOTA_MT')
df_train = df_train[cols]
df_train = df_train.select_dtypes(exclude=['object'])
df_test = df_test.select_dtypes(exclude=['object'])


In [164]:
cols = df_train.columns
num_cols = df_train._get_numeric_data().columns
encodar = list(set(cols) - set(num_cols))
le = LabelEncoder()
encoded = df_train[encodar].apply(lambda col : le.fit_transform(col))
df_train.update(encoded)
cols = df_test.columns
num_cols = df_test._get_numeric_data().columns
encodar = list(set(cols) - set(num_cols))
le = LabelEncoder()
encoded = df_test[encodar].apply(lambda col : le.fit_transform(col))
df_test.update(encoded)

In [165]:
cols = list(df_test.columns)
cols.append('NU_NOTA_MT')
df_train = df_train[cols]

In [166]:
df_train.head()

,CO_UF_RESIDENCIA,NU_IDADE,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,TP_DEPENDENCIA_ADM_ESC,...,NU_NOTA_LC,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,NU_NOTA_MT
0,43,24,1,1,1,4,1,0.0,0,0.0,...,581.2,1,1.0,120.0,120.0,120.0,80.0,80.0,520.0,399.4
1,23,17,3,1,2,0,2,1.0,0,2.0,...,599.0,1,1.0,140.0,120.0,120.0,120.0,80.0,580.0,459.8
2,23,21,3,1,3,0,1,0.0,0,0.0,...,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,33,25,0,1,1,9,1,0.0,0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,13,28,2,1,1,4,1,0.0,0,0.0,...,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [167]:
df_test.head()

,CO_UF_RESIDENCIA,NU_IDADE,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,TP_DEPENDENCIA_ADM_ESC,...,NU_NOTA_CH,NU_NOTA_LC,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO
0,41,22,3,1,1,5,1,0.0,0,0.0,...,443.5,431.8,0,1.0,120.0,80.0,80.0,100.0,40.0,420.0
1,21,26,3,1,1,8,1,0.0,0,0.0,...,491.1,548.0,1,1.0,120.0,120.0,120.0,120.0,100.0,580.0
2,23,21,1,1,2,0,2,3.0,0,2.0,...,622.7,613.6,0,1.0,80.0,40.0,40.0,80.0,80.0,320.0
3,15,27,3,1,1,8,1,0.0,0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,41,18,1,1,2,0,2,1.0,0,2.0,...,492.6,571.4,1,1.0,100.0,80.0,60.0,80.0,0.0,320.0


In [168]:
def profile(df):
    return pd.DataFrame({
        'dtypes':df.dtypes,
        'Nan': df.isna().sum(),
        'nunique':df.nunique()
        
    }).T

profile(df_train)

,CO_UF_RESIDENCIA,NU_IDADE,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,TP_DEPENDENCIA_ADM_ESC,...,NU_NOTA_LC,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,NU_NOTA_MT
dtypes,int64,int64,int64,int64,int64,int64,int64,float64,int64,float64,...,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64
Nan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
nunique,27,55,6,5,4,11,4,4,2,5,...,2774,2,10,15,13,12,14,14,53,3406


In [169]:
df_train.corrwith(df_train.NU_NOTA_MT).sort_values()

NU_IDADE                 -0.240874
TP_ANO_CONCLUIU          -0.191553
TP_LINGUA                -0.111428
TP_COR_RACA              -0.070239
TP_NACIONALIDADE         -0.033130
IN_GESTANTE              -0.028129
IN_IDOSO                 -0.009061
IN_BAIXA_VISAO           -0.006546
IN_SURDEZ                -0.002617
IN_SABATISTA             -0.002457
CO_UF_RESIDENCIA          0.002613
IN_DISLEXIA               0.005259
IN_DISCALCULIA            0.008392
TP_ST_CONCLUSAO           0.023736
IN_TREINEIRO              0.078804
TP_ENSINO                 0.083482
TP_DEPENDENCIA_ADM_ESC    0.185663
TP_ESCOLA                 0.189084
TP_STATUS_REDACAO         0.564273
NU_NOTA_COMP5             0.722275
NU_NOTA_COMP3             0.854946
NU_NOTA_COMP2             0.862709
NU_NOTA_COMP4             0.866827
TP_PRESENCA_CN            0.874764
TP_PRESENCA_CH            0.874764
NU_NOTA_REDACAO           0.875929
NU_NOTA_COMP1             0.882746
NU_NOTA_CH                0.903608
NU_NOTA_CN          

In [170]:
y = df_train['NU_NOTA_MT']
X = df_train.drop(['NU_NOTA_MT'],1)
y = y.values.reshape(-1,1)

In [171]:
scalerx= StandardScaler()
X = scalerx.fit_transform(X)


In [172]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

In [173]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (10297, 32)
Training Labels Shape: (10297, 1)
Testing Features Shape: (3433, 32)
Testing Labels Shape: (3433, 1)


In [174]:
regr = RandomForestRegressor(n_estimators=150,criterion='mse',random_state=0,n_jobs=100)

In [175]:
regr.fit(X_train,y_train)

C:\Users\Weslley\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=150, n_jobs=100, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [176]:
print(mean_squared_error(regr.predict(X_test),y_test))
print(mean_absolute_error(regr.predict(X_test),y_test))
regr.score(X_test,y_test)

4352.694791276176
45.1286091853578


0.9176640428073132

In [177]:

X_final = df_test
X_final = scalerx.transform(X_final)
previsao = regr.predict(X_final).reshape(-1,1)

In [178]:
df_out['NU_NOTA_MT'] = previsao

In [179]:
df_out.to_csv('answer.csv',index=False)